In [138]:
import time
import argparse as ap
import cv2
import imutils 
import numpy as np
import os
from sklearn.svm import LinearSVC
from sklearn.externals import joblib
from scipy.cluster.vq import *
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from sklearn.externals import joblib
def imlist(path):
    """
    The function imlist returns all the names of the files in 
    the directory path supplied as argument to the function.
    """
    return [os.path.join(path, f) for f in os.listdir(path)]

In [ ]:
#!/usr/local/bin/python2.7

# Get the path of the training set
#parser = ap.ArgumentParser()
#parser.add_argument("-t", "--trainingSet", help="Path to Training Set", required="True")
#args = vars(parser.parse_args())
t1 = time.time()
# Get the training classes names and store them in a list
train_path = 'hw4_data/hw4_data/train/'
training_names = os.listdir(train_path)

# Get all the path to the images and save them in a list
# image_paths and the corresponding label in image_paths
image_paths = []
image_classes = []
class_id = 0
for training_name in training_names:
    dir = os.path.join(train_path, training_name)
    class_path = imlist(dir)
    image_paths+=class_path
    image_classes+=[class_id]*len(class_path)
    class_id+=1

# Create feature extraction and keypoint detector objects
#fea_det = cv2.FeatureDetector_create("SIFT")
#des_ext = cv2.DescriptorExtractor_create("SIFT")
sift = cv2.xfeatures2d.SIFT_create()
#(kps, descs) = sift.detectAndCompute(gray, None)
# List where all the descriptors are stored
des_list = []
#print (image_paths)
for index,image_path in enumerate(image_paths):
    #print (index, image_path)
    im = cv2.imread(image_path,0)
    im = cv2.resize(im,(256,256))
    (kpts, des) = sift.detectAndCompute(im, None)
    #kpts = fea_det.detect(im)
    #kpts, des = des_ext.compute(im, kpts)
    des_list.append((image_path, des))   
print('done')       

In [ ]:
# Stack all the descriptors vertically in a numpy array

#print(des_list[1:])
descriptors = des_list[0][1]
for image_path, descriptor in des_list[1:]:
    #print(descriptors,descriptor)
    #if(descriptor is None):
        #continue
    descriptors = np.vstack((descriptors, descriptor))  
print('ok')
# Perform k-means clustering
k =100
voc, variance = kmeans(descriptors, k, 1)

#voc = KMeans(n_clusters=k, random_state=0).fit(descriptors)
#joblib.dump((des_list,image_paths,voc, variance), "k1000.pkl", compress=3)    
print('done')

In [139]:
#train_path = 'hw4_data/hw4_data/train/'
#training_names = os.listdir(train_path)
#
## Get all the path to the images and save them in a list
## image_paths and the corresponding label in image_paths
#image_paths = []
#image_classes = []
#class_id = 0
#for training_name in training_names:
#    dir = os.path.join(train_path, training_name)
#    class_path = imlist(dir)
#    image_paths+=class_path
#    image_classes+=[class_id]*len(class_path)
#    class_id+=1
## Create feature extraction and keypoint detector objects
##fea_det = cv2.FeatureDetector_create("SIFT")
##des_ext = cv2.DescriptorExtractor_create("SIFT")
#sift = cv2.xfeatures2d.SIFT_create()
##(kps, descs) = sift.detectAndCompute(gray, None)
## List where all the descriptors are stored
#des_list = []
##print (image_paths)
#for index,image_path in enumerate(image_paths):
#    #print (index, image_path)
#    im = cv2.imread(image_path,0)
#    im = cv2.resize(im,(800,800))
#    (kpts, des) = sift.detectAndCompute(im, None)
#    #kpts = fea_det.detect(im)
#    #kpts, des = des_ext.compute(im, kpts)
#    des_list.append((image_path, des))   
#print('done')   

done


In [140]:
#a,b,voc, variancec = joblib.load("k1000.pkl")

#k=1000
# Calculate the histogram of features

im_features = np.zeros((len(image_paths), k), "float32")
for i in range(len(image_paths)):
    #if (des_list[i][1] is None):
        #continue
    words, distance = vq(des_list[i][1],voc)
    for w in words:
        im_features[i][w] += 1
    #im_features[i] = (im_features[i] - np.mean(im_features[i])) / np.std(im_features[i])
    
# Perform Tf-Idf vectorization
nbr_occurences = np.sum( (im_features > 0) * 1, axis = 0)
idf = np.array(np.log((1.0*len(image_paths)+1) / (1.0*nbr_occurences + 1)), 'float32')

# Scaling the words
stdSlr = StandardScaler().fit(im_features)
im_features = stdSlr.transform(im_features)

# Train the Linear SVM
clf = SVC(C=0.01, cache_size=1000,  coef0=0.0,
   decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=True, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
#clf = LinearSVC(C=0.001, class_weight=None, dual=True, fit_intercept=True,
#     intercept_scaling=1, loss='squared_hinge', max_iter=50000,
#     multi_class='ovr', penalty='l2', random_state=0, tol=0.0001,
#     verbose=0)
#clf = OneVsRestClassifier()
clf.fit(im_features, np.array(image_classes))

# Save the SVM
#joblib.dump((clf, training_names, stdSlr, k, voc), "bof.pkl", compress=3)    

SVC(C=0.01, cache_size=1000, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [141]:
#print(image_classes)

In [142]:
#!/usr/local/bin/python2.7

import argparse as ap
import cv2
import imutils 
import numpy as np
import os
from sklearn.svm import LinearSVC
from sklearn.externals import joblib
from scipy.cluster.vq import *


classes_names = training_names


import glob
image_paths = glob.glob("hw4_data/hw4_data/test/**/*")


sift = cv2.xfeatures2d.SIFT_create()
# List where all the descriptors are stored
des_list = []

for image_path in image_paths:
    im = cv2.imread(image_path)
    im = cv2.resize(im,(1000,1000))
    
    if (im is None):
        print ("No such file {}\nCheck if the file exists".format(image_path))
        exit()
    (kpts, des) = sift.detectAndCompute(im, None)    
    des_list.append((image_path, des))   
    
# Stack all the descriptors vertically in a numpy array
descriptors = des_list[0][1]
for image_path, descriptor in des_list[0:]:
    descriptors = np.vstack((descriptors, descriptor)) 

# 
test_features = np.zeros((len(image_paths), k), "float32")
for i in range(len(image_paths)):
    words, distance = vq(des_list[i][1],voc)
    for w in words:
        test_features[i][w] += 1
    #test_features[i] = (test_features[i] - np.mean(test_features[i])) / np.std(test_features[i])
# Perform Tf-Idf vectorization
nbr_occurences = np.sum( (test_features > 0) * 1, axis = 0)
idf = np.array(np.log((1.0*len(image_paths)+1) / (1.0*nbr_occurences + 1)), 'float32')

# Scale the features
stdSlr = StandardScaler().fit(test_features)
test_features = stdSlr.transform(test_features)

# Perform the predictions
predictions =  [classes_names[i] for i in clf.predict(test_features)]
print(predictions[0:10])


['Bedroom', 'Office', 'Kitchen', 'Bedroom', 'Office', 'Office', 'Bedroom', 'Bedroom', 'Bedroom', 'Office']


In [143]:
#print(idf)

In [144]:
total = 0
for i in range(15):
    #print(classes_names[i])
    count = 0.
    for j in range(10):
        if classes_names[i] == predictions[10*i+j]:
            count += 1
    print(classes_names[i],count/10.)
    total += count
print("acc", total/150.)
    
    #print(predictions[0+10*i:10+i*10])

Bedroom 0.5
Coast 0.7
Forest 0.6
Highway 0.8
Industrial 0.1
InsideCity 0.7
Kitchen 0.6
LivingRoom 0.3
Mountain 0.7
Office 0.8
OpenCountry 0.6
Store 0.5
Street 0.6
Suburb 0.9
TallBuilding 0.4
acc 0.5866666666666667


In [145]:
#print(clf.predict(test_features))
label = [[i]*10 for i in range(15)]
label = np.asarray(label)
label = np.reshape(label,150)
#print(label)

In [146]:
import pandas as pd
from sklearn.metrics import confusion_matrix
print ('Linear SVM')
#print ('No. of test instances:',len(XTEST),len(YTEST))
### Accuracy Report ###
#hybridpred=hybrid_classifier.predict(XTEST)
print ('Overall accuracy:',total/150)
#print (classification_report(YTEST, hybridpred, target_names=category_names))
print ('Confusion matrix:\n')
pd.DataFrame(confusion_matrix(clf.predict(test_features),label),
             columns=['Bedroom', 'Coast', 'Forest', 'Highway', 'Industrial', 'InsideCity', 'Kitchen', 'LivingRoom', 'Mountain', 'Office', 'OpenCountry', 'Store', 'Street', 'Suburb', 'TallBuilding'],
               index=['Bedroom', 'Coast', 'Forest', 'Highway', 'Industrial', 'InsideCity', 'Kitchen', 'LivingRoom', 'Mountain', 'Office', 'OpenCountry', 'Store', 'Street', 'Suburb', 'TallBuilding'])

Linear SVM
Overall accuracy: 0.5866666666666667
Confusion matrix:



,Bedroom,Coast,Forest,Highway,Industrial,InsideCity,Kitchen,LivingRoom,Mountain,Office,OpenCountry,Store,Street,Suburb,TallBuilding
Bedroom,5,0,0,0,2,0,1,0,0,0,0,1,0,0,0
Coast,0,7,0,0,1,0,0,0,1,0,2,0,0,0,0
Forest,0,0,6,0,0,0,0,0,1,0,1,1,1,0,1
Highway,0,0,0,8,0,0,0,0,0,0,1,0,0,1,0
Industrial,0,0,0,1,1,0,1,1,0,0,0,0,0,0,2
InsideCity,0,0,0,0,0,7,0,0,0,0,0,1,1,0,0
Kitchen,1,0,0,0,3,0,6,2,0,0,0,1,1,0,0
LivingRoom,0,0,0,0,0,1,0,3,0,2,0,0,0,0,0
Mountain,0,0,1,0,0,0,0,0,7,0,0,1,0,0,0
Office,4,0,0,1,0,0,2,3,0,8,0,0,0,0,0
